# 1) Loading and cleaning the data

In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

#print('Before dropping Boroughs ='.join(df.shape))
df_wikipedia= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

#there are no Postal codes repeated so we dont need to concatinate the Neighbourhoods.

df_wikipedia.shape

(103, 3)

# 2) Loading LAtitude and logitude

In [3]:
!pip install geocoder
import geocoder

In [4]:
!wget -q -O 'zipcode.csv' http://cocl.us/Geospatial_data

In [5]:
#df.head(1)
#geocoder.google('M3A, Toronto, Ontario')

df1_zip = pd.read_csv('zipcode.csv')
df1_zip.head()

df_merged = pd.merge(df_wikipedia,df1_zip, on=['Postal Code'])

df_merged.head()

print(df_merged.size)

#geocoder.google('{}, Toronto, Ontario'.format(df['Postal Code'].iloc(0)))

515


In [6]:
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Filtering data frames to contain Boroughs that have the word Toronto in them

In [7]:
df_final=df_merged[df_merged['Borough'] .str.contains('Toronto')].reset_index(drop=True)
df_final.head()
print(df_final.shape)

(39, 5)


# Now talk to Foursquare API to explore each of these locations and find categories of places around each within a radius of 10 M

In [8]:
import requests
from pandas.io.json import json_normalize


fourSqAPIURL = "https://api.foursquare.com/v2/venues/explore"
CLIENT_ID ="NINRXL4F12KA0T5Y40SQD1CGLLM2FK1XGE1YZBYMVQINX45V"
CLIENT_SECRET= "4NSTMXF2D5VWNSAMNRUMP3RC1W4NHOLDSVOLK3UX1BXBAZD5"

    

In [9]:
def fetch_venue_Details(postalcode,latitude,longitude,v_list):
    new_venue = {'Postal Code':postalcode,'Latitude':latitude, 'Longitude':longitude,'Type':[],'Name':[]}

    new_venue["Name"] = v_list["venue"]["name"]
    new_venue["Type"] = v_list["venue"]["categories"][0]['pluralName']
    

    new_venues = pd.DataFrame(new_venue, index =[0])
    
  
   # new_venues = new_venues.append(new_venue,ignore_index=True)
    #print('after--- '+ str(new_venues))
    return new_venues


In [10]:

all_venues= pd.DataFrame()


for ind in df_final.index:

   
    lat = str(df_final['Latitude'][ind])
    lng = str(df_final['Longitude'][ind])

    
 
    payload = {'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET, 'll': lat+','+lng , 'radius':200, 'v':'20180323'}

    resp = requests.get(fourSqAPIURL,params=payload)

    data = resp.json()
    
    for items in data["response"]["groups"][0]["items"]:
    
        all_venues = all_venues.append(fetch_venue_Details(df_final['Postal Code'][ind],lat,lng,items))


before---   Postal Code    Latitude    Longitude      Type              Name
0         M5A  43.6542599  -79.3606359  Bakeries  Roselle Desserts
before---   Postal Code    Latitude    Longitude          Type           Name
0         M5A  43.6542599  -79.3606359  Coffee Shops  Tandem Coffee
before---   Postal Code    Latitude    Longitude             Type                Name
0         M5A  43.6542599  -79.3606359  Breakfast Spots  Morning Glory Cafe
before---   Postal Code    Latitude    Longitude  Type                 Name
0         M5A  43.6542599  -79.3606359  Spas  Body Blitz Spa East
before---   Postal Code    Latitude    Longitude                     Type  \
0         M5A  43.6542599  -79.3606359  Gyms or Fitness Centers   

                 Name  
0  The Extension Room  
before---   Postal Code    Latitude    Longitude             Type  \
0         M5A  43.6542599  -79.3606359  History Museums   

                       Name  
0  Enoch Turner Schoolhouse  
before---   Postal Code 

In [16]:


all_venues = all_venues.reset_index(drop=True)


all_venues
#venues.append()


#for venue in data["response"]["groups"][0]["items"][0].items():
 #   venue

,Postal Code,Latitude,Longitude,Type,Name
0,M5A,43.6542599,-79.3606359,Bakeries,Roselle Desserts
1,M5A,43.6542599,-79.3606359,Coffee Shops,Tandem Coffee
2,M5A,43.6542599,-79.3606359,Breakfast Spots,Morning Glory Cafe
3,M5A,43.6542599,-79.3606359,Spas,Body Blitz Spa East
4,M5A,43.6542599,-79.3606359,Gyms or Fitness Centers,The Extension Room
5,M5A,43.6542599,-79.3606359,History Museums,Enoch Turner Schoolhouse
6,M7A,43.6623015,-79.3894938,Thai Restaurants,Thai Express
7,M7A,43.6623015,-79.3894938,Lakes,Niagara on the lake
8,M5B,43.6571618,-79.37893709999999,Clothing Stores,UNIQLO ユニクロ
9,M5B,43.6571618,-79.37893709999999,Tea Rooms,DAVIDsTEA


In [39]:
neighbourhood_analysis = pd.get_dummies(all_venues,prefix='',prefix_sep='',columns=['Type'])

In [40]:
neighbourhood_analysis.head()

,Postal Code,Latitude,Longitude,Name,Accessories Stores,Adult Boutiques,American Restaurants,Arepa Restaurants,Art Galleries,Arts & Crafts Stores,...,Theaters,Theme Restaurants,Thrift / Vintage Stores,Toy / Game Stores,Trails,Transportation Services,Vegetarian / Vegan Restaurants,Vietnamese Restaurants,Wine Bars,Yoga Studios
0,M5A,43.6542599,-79.3606359,Roselle Desserts,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,43.6542599,-79.3606359,Tandem Coffee,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,43.6542599,-79.3606359,Morning Glory Cafe,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,43.6542599,-79.3606359,Body Blitz Spa East,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,43.6542599,-79.3606359,The Extension Room,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
mean_data = neighbourhood_analysis.set_index(['Postal Code'])
mean_data = mean_data.groupby(level='Postal Code').mean()

In [46]:
mean_data.head()

,Accessories Stores,Adult Boutiques,American Restaurants,Arepa Restaurants,Art Galleries,Arts & Crafts Stores,Asian Restaurants,Auto Workshops,BBQ Joints,Bakeries,...,Theaters,Theme Restaurants,Thrift / Vintage Stores,Toy / Game Stores,Trails,Transportation Services,Vegetarian / Vegan Restaurants,Vietnamese Restaurants,Wine Bars,Yoga Studios
Postal Code,,,,,,,,,,,,,,,,,,,,,
M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M4P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
